# Implementing a 5-Qubit Quantum Fourier Transform (QFT)

The Quantum Fourier Transform (QFT) is a key operation in quantum computing, playing a central role in several quantum algorithms, such as Shor's algorithm for integer factorization and quantum phase estimation. The QFT transforms the amplitudes of a quantum state, changing its basis and making certain computations more efficient than classical methods.

## Steps to Perform a 5-Qubit Quantum Fourier Transform:

1. **Initial State**:
   - Begin with a 5-qubit quantum register initialized in the computational basis state \( |0\rangle \). This means all qubits are initially in the state \( |00000\rangle \).
   $$
   |00000\rangle
   $$

2. **Apply Hadamard Gates**:
   - Apply a Hadamard gate to each qubit. The Hadamard gate creates a superposition of the basis states:
   $$
   (H \otimes H \otimes H \otimes H \otimes H) |00000\rangle = \frac{1}{\sqrt{32}} \sum_{x=0}^{31} |x\rangle
   $$
   - The summation over \( x \) represents all possible 5-bit binary states, creating a superposition of all 32 basis states.

3. **Apply Controlled Phase Rotations**:
   - Apply controlled-phase rotations to introduce quantum interference, which is a crucial step for the Fourier transformation. These controlled rotations are defined as follows:
     - **C-R1**: Rotate qubit 1 by \( \frac{\pi}{2} \)
     - **C-R2**: Rotate qubit 2 by \( \frac{\pi}{4} \)
     - **C-R3**: Rotate qubit 3 by \( \frac{\pi}{8} \)
     - **C-R4**: Rotate qubit 4 by \( \frac{\pi}{16} \)
   - These rotations help to transform the quantum state in a way that mimics the behavior of a discrete Fourier transform.

4. **Swap Qubits**:
   - Perform swap operations to rearrange the qubits into the correct order. The QFT algorithm requires the qubits to be in a specific order to achieve the correct output state.
   - Swap gates ensure that the qubits are positioned in the correct order after the rotations.

5. **Measurement**:
   - Measure each qubit to collapse the quantum state into classical bit strings. The results of the measurements represent the amplitudes in the new basis, corresponding to the Fourier-transformed values of the input state.
   - The probability distribution of the measurement outcomes should ideally match the QFT of the original state.

   **Important Note**: Implementing the QFT on real quantum hardware can be challenging due to errors and noise. Practical implementations often require error correction techniques and careful calibration of quantum gates.

---

## Implementation Steps:

1. **Import the Necessary Libraries**:
   - First, we import the required libraries for quantum circuit creation and execution.

2. **Create the Quantum Circuit**:
   - Next, we create a quantum circuit with 5 qubits and corresponding classical registers for measurement.

3. **Apply the Hadamard Gates**:
   - Apply Hadamard gates to each qubit in the circuit to create a superposition.

4. **Apply Controlled Phase Shift Gates (QFT)**:
   - Apply the controlled phase shifts and the QFT operation to the qubits.

5. **Measure the Qubits**:
   - Finally, measure the qubits to get the output in classical bit strings.


In [1]:
!pip install qiskit


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install qiskit-aer


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install pylatexenc


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install pylatexenc


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer  # Corrected import for Aer simulator
from qiskit.circuit.library import QFT
import numpy as np

# Use Aer simulator backend
backend = Aer.get_backend('qasm_simulator')

# Define pi constant
pi = np.pi

# Create Quantum and Classical Registers
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

# Create Quantum Circuit
circuit = QuantumCircuit(q, c)

# Apply some X gates
circuit.x(q[4])
circuit.x(q[2])
circuit.x(q[0])

# Apply Quantum Fourier Transform (QFT)
circuit.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=False, insert_barriers=False, name='qft'), q)

# Measure the quantum register
circuit.measure(q, c)

# Draw the circuit
circuit.draw(output='mpl', filename='qft1.png')

print(circuit)

# Transpile the circuit for the Aer backend
transpiled_circuit = transpile(circuit, backend)

# Execute the circuit
job = backend.run(transpiled_circuit, shots=1000)

# Get the results
result = job.result()
counts = result.get_counts()

# Display the QFT output
print("\nQFT Output")
print("-------------")
print(counts)

# Second Circuit: Apply Inverse QFT
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

circuit = QuantumCircuit(q, c)

# Apply the same X gates as before
circuit.x(q[4])
circuit.x(q[2])
circuit.x(q[0])

# Apply Quantum Fourier Transform (QFT)
circuit.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=True, insert_barriers=False, name='inverse_qft'), q)

# Measure the quantum register
circuit.measure(q, c)

# Draw the second circuit
circuit.draw(output='mpl', filename='qft2.png')

print(circuit)

# Transpile the second circuit for the Aer backend
transpiled_circuit = transpile(circuit, backend)

# Execute the second circuit
job = backend.run(transpiled_circuit, shots=1000)

# Get the results for the second job
result = job.result()
counts = result.get_counts()

# Display the QFT with inverse QFT output
print("\nQFT with inverse QFT Output")
print("------------------------------")
print(counts)


     ┌───┐┌──────┐┌─┐            
q_0: ┤ X ├┤0     ├┤M├────────────
     └───┘│      │└╥┘┌─┐         
q_1: ─────┤1     ├─╫─┤M├─────────
     ┌───┐│      │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 qft ├─╫──╫─┤M├──────
     └───┘│      │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├─╫──╫──╫─┤M├───
     ┌───┐│      │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ X ├┤4     ├─╫──╫──╫──╫─┤M├
     └───┘└──────┘ ║  ║  ║  ║ └╥┘
c: 5/══════════════╩══╩══╩══╩══╩═
                   0  1  2  3  4 

QFT Output
-------------
{'00011': 26, '01111': 26, '10011': 26, '11100': 36, '10010': 34, '10101': 31, '11110': 31, '01001': 28, '00101': 29, '10001': 34, '11010': 31, '00001': 30, '00110': 16, '00010': 32, '11101': 38, '10111': 28, '10100': 36, '10000': 40, '11011': 37, '11111': 36, '11000': 29, '01000': 34, '01100': 31, '11001': 38, '00000': 25, '01110': 35, '00111': 31, '01011': 34, '01010': 33, '10110': 35, '01101': 28, '00100': 22}
     ┌───┐┌──────────────┐┌─┐            
q_0: ┤ X ├┤0             ├┤M├────────────
     └───┘│              │└╥┘┌─